In [13]:
import rdflib
from rdflib.graph import Graph, URIRef
from SPARQLWrapper import SPARQLWrapper, XML


In [14]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

construct_query= """
PREFIX ma: <http://www.semanticweb.org/dibah/ontologies/2024/3/untitled-ontology-11#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX wd: <http://www.wikidata.org/entity/>

CONSTRUCT {
    ?book rdf:type ma:Book .
    ?book ma:title ?name .
    ?book ma:written_by ?author .
    ?author rdf:type ma:Author .
    ?book ma:published_by ?publisher .
    ?publisher rdf:type ma:publisher .
    ?book ma:has_genre ?genre .
    ?genre rdf:type ma:literaryGenre .
    ?book ma:cover_designed_by ?coverArtist .
    ?book ma:numberOfPages ?numberOfPages .
    ?book ma:isbn ?isbn .
    ?book ma:releaseDate ?releaseDate .
    ?book ma:has_publish_country ?country .
    ?country rdf:type ma:Country .
    ?book ma:has_language ?lang .
    ?lang rdf:type ma:Language .
    ?book ma:has_character ?Character .
    ?Character rdf:type ma:Character .      
  }
WHERE {
  ?book wdt:P31 wd:Q571 .
  ?book wdt:P1476 ?name .
  OPTIONAL { ?book wdt:P407 ?lang .}
  OPTIONAL { ?book wdt:P50 ?author . }
  OPTIONAL { ?book wdt:P136 ?genre . }
  OPTIONAL { ?book wdt:P123 ?publisher . }
  ?book wdt:P577 ?releaseDate .
  OPTIONAL { ?book wdt:P495 ?Country . }
  OPTIONAL { ?book wdt:P736 ?coverArtist . }
  OPTIONAL { ?book wdt:P674 ?Character . }

  FILTER (LANG(?name) = 'en') .
}"""

sparql.setQuery(construct_query)
sparql.setReturnFormat(XML)

graph_id = URIRef("http://www.semanticweb.org/store/book")
g = Graph(identifier=graph_id)

print("I might take some time, bear with me...")

g = sparql.query().convert()

g.parse("book_dbpedia.owl")

g.serialize("book_bonus.owl", format="xml")

print("...All done!")
print("")

I might take some time, bear with me...
...All done!

